In [1]:
import dask.dataframe as pd
from dask.delayed import delayed
import numpy as np
from decimal import Decimal,ROUND_HALF_UP

In [2]:
df = pd.read_csv('/Users/javedakhtar/Documents/JAVED/PROGRAMMING/PYTHON/0Projects/complex_raw_org_p5.csv').compute()

In [3]:
def to_decimal(data):
    if data=='xx':
        return 'xx'
    return Decimal(data)

In [4]:
print(df.shape)
df

(36018000, 4)


,real,imaginary,power,iteration
0,xx,xx,xx,xx
1,0.01000000000000000020816681711721685132943093...,-10,2,1
2,-99.99989999999999999999583666,-0.2000000000000000041633363424,2,2
3,9999.940000009999999997501998,39.99996000000000083266477048,2,3
4,99997200.00699999719988352714,799994.4000056000158529956498,2,4
...,...,...,...,...
977203,2.031360806524753570556640625E+28,-973464764654636383056640625,5,4
977204,-1.204156818624106165161998352E+84,8.324511947225180231208903029E+84,5,5
977205,-5.406563914314278307174059474E+254,2.485891486463213907158856505E+254,5,6
977206,2.026328297007109247747504382E+764,-5.780678626060487185945127585E+763,5,7


In [5]:
# index_xx = df[df['real']=='xx'].index
mask = df['real'] != 'xx'
df = df.loc[mask]
df = df.reset_index(drop=True)
print(df.shape)
df

(32016000, 4)


,real,imaginary,power,iteration
0,0.01000000000000000020816681711721685132943093...,-10,2,1
1,-99.99989999999999999999583666,-0.2000000000000000041633363424,2,2
2,9999.940000009999999997501998,39.99996000000000083266477048,2,3
3,99997200.00699999719988352714,799994.4000056000158529956498,2,4
4,9998800018199919.920078741394,159994400043679.8889304337581,2,5
...,...,...,...,...
32015995,2.031360806524753570556640625E+28,-973464764654636383056640625,5,4
32015996,-1.204156818624106165161998352E+84,8.324511947225180231208903029E+84,5,5
32015997,-5.406563914314278307174059474E+254,2.485891486463213907158856505E+254,5,6
32015998,2.026328297007109247747504382E+764,-5.780678626060487185945127585E+763,5,7


In [ ]:
def apply_to_decimal(column):
    return column.map(to_decimal)

df['real'] = apply_to_decimal(df['real'])
df['imaginary'] = apply_to_decimal(df['imaginary'])
df

In [ ]:
df = df.replace(to_replace=Decimal('-0.0'),
           value=Decimal('0.0'))
df[df['imaginary']==Decimal('0.0')]

In [ ]:
quadrant_rule = {(True,True):'A',(False,True):'B',(False,False):'C',(True,False):'D'}

In [ ]:
def check_quadrant(real, imaginary):
    if real == 0 and imaginary == 0:
        return 'O'
    elif real == 0 and imaginary != 0:
        return 'Y'
    elif real != 0 and imaginary == 0:
        return 'X'
    else:
        return quadrant_rule[(real > 0, imaginary > 0)]

In [ ]:
df['quadrant'] = df.apply(lambda row: check_quadrant(row['real'],row['imaginary']),axis=1)
df['sequence'] = np.nan
print(df.shape)
df

In [ ]:
for i,row in df.iterrows():
    if row['iteration'] == '1':
        sequence = ''
        for j in range(0,8):
#             print(df.iloc[i+j]['quadrant'],i)
            sequence += df.iloc[i+j]['quadrant']
        print(sequence,i)
        df.at[i,'sequence'] = sequence
        

In [ ]:
df = df[df['iteration']=='1'].drop('iteration',axis=1).drop('quadrant',axis=1)

In [ ]:
def round_decimal(num, decimal_places=2):
    return num.quantize(Decimal('.1') ** decimal_places,
                        rounding=ROUND_HALF_UP)

# apply the function to the column
df['real']=df['real'].map(round_decimal)
df['imaginary']=df['imaginary'].map(round_decimal)
df.tail(20)

In [ ]:
df[df['real']==df['imaginary']].head(20)

In [ ]:
df.to_csv('complex_org_p5.csv',index=False)

In [ ]:
df[df['sequence'].astype(str).str[:1]=='B']

In [ ]:
df['sequence'].unique().size